In [ ]:
import os

filename="outputs/ComfyUI_00309_.png"

file=os.path.basename(filename)
file

In [ ]:
from llama_cpp import Llama
import json

model_id="/Users/janghyeonbin/ai-companion/models/llm/gguf/Meta-Llama-3.1-8B-Instruct.Q4_K_M.gguf"
model=Llama(model_id)

chat="안녕?"
response = model.create_chat_completion(
    messages=[
        {"role": "system", "content": "당신은 유용한 AI 비서입니다."},
        {"role": "user", "content": chat}
    ]
)

response["choices"][0]["message"]["content"]

In [ ]:
from diffusers import AutoPipelineForText2Image


In [ ]:
pipeline = AutoPipelineForText2Image.from_pretrained("runwayml/stable-diffusion-v1-5")

In [ ]:
from huggingface_hub import snapshot_download

In [ ]:
snapshot_download(repo_id="faridlazuarda/valadapt-llama-3.1-8B-it-korean", local_dir="./models/llm/loras/valadapt-llama-3.1-8B-it-korean")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path="/Volumes/EXDATA/models/LLM/transformers/llama/meta-llama__Llama-3.1-8B-Instruct"
lora_path="/Users/janghyeonbin/ai-companion/models/llm/loras/valadapt-llama-3.1-8B-it-korean"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype="auto")
model.load_adapter(lora_path)

In [1]:
import cv2
import imutils
import pytesseract

In [2]:
image=cv2.imread('./outputs/ComfyUI_00318_.png')

In [3]:
image.shape

(1344, 768, 3)

In [34]:
from PIL import Image, ImageOps

In [37]:
original_image=Image.open('ComfyUI_00318_.png').convert('RGBA')
original_image.show()

python(22982) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(22983) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [38]:
image=Image.open('layer_0.png').convert('RGBA')
image.show()

python(22984) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(22985) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [39]:
new_alpha=image.getchannel('A')
new_alpha.show()

python(23021) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(23023) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [40]:
new_alpha=ImageOps.invert(new_alpha)
new_alpha.show()

python(23070) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(23071) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [44]:
new_mask=Image.new("RGBA", image.size)
new_mask.show()

python(23358) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(23359) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [45]:
new_mask.putalpha(new_alpha)
new_mask.show()

python(23360) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(23361) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
